In [1]:
# https://api.mongodb.com/python/current/tutorial.html
import pprint
from pymongo import MongoClient
import pandas as pd
import requests as req 
import json
import math
import re
client = MongoClient('mongodb://localhost:27017/')
db = client.ejercicio

In [2]:
# {$and:[{deadpooled_year:null}, {number_of_employees:{$ne:null}}, {"offices.latitude":{$ne:null}}, {"offices.latitude":{$exists:true}}]}

In [3]:
empresas= db.companhias.find({"$and":[{"deadpooled_year":None}, {"number_of_employees":{"$ne":None}},{"total_money_raised":{"$ne":None}}, {"offices.latitude":{"$ne":None}}, {"offices.latitude":{"$exists":"true"}}]})

In [4]:
df=pd.DataFrame(empresas)[["name","category_code","number_of_employees","founded_year", "total_money_raised","offices"]]

In [5]:
df.shape

(4839, 6)

In [6]:
df.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Digg,news,60,2004.0,$45M,"[{'description': None, 'address1': '135 Missis..."
1,Geni,web,18,2006.0,$16.5M,"[{'description': 'Headquarters', 'address1': '..."
2,Fox Interactive Media,web,0,1979.0,$0,"[{'description': '', 'address1': '407 N Maple ..."
3,Scribd,news,50,2007.0,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant..."
4,MeetMoi,social,15,2007.0,$5.58M,"[{'description': None, 'address1': '', 'addres..."


In [7]:
df1=df.copy()

In [8]:
res = req.get("https://api.exchangeratesapi.io/latest?base=USD").json()

In [9]:
def cambiar(string):
    if string.startswith("$"):
        string=string.replace("$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= float(string)*1000000
        elif string.endswith("k"):
            string=string.replace("k","")
            string= float(string)*1000
        elif string.endswith("B"):
            string=string.replace("B","")
            string= float(string)*1000000000
        else:
            string=float(string)
    
    elif string.startswith("€"):
        string=string.replace("€","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["EUR"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["EUR"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["EUR"]
        else:
            string=float(string)
            
    elif string.startswith("£"):
        string=string.replace("£","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["GBP"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["GBP"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["GBP"]
        else:
            string=float(string)

    elif string.startswith("¥"):
        string=string.replace("¥","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["JPY"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["JPY"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["JPY"]
        else:
            string=float(string)
      
    elif string.startswith("C$"):
        string=string.replace("C$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["CAD"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["CAD"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["CAD"]
        else:
            string=float(string)
            
    elif string.startswith("kr"):
        string=string.replace("kr","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["SEK"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["SEK"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["SEK"]
        else:
            string=float(string)
        
    return string

In [10]:
df1["total_money_raised"]=df1["total_money_raised"].apply(cambiar)

In [11]:
df1["total_money_raised"].value_counts()

0.000000e+00    3559
1.000000e+06      36
5.000000e+05      24
1.000000e+07      22
3.000000e+06      19
1.000000e+05      18
2.500000e+05      17
6.000000e+05      17
4.000000e+06      15
5.000000e+06      15
2.500000e+06      12
1.500000e+05      10
2.000000e+06      10
1.100000e+07       9
3.500000e+05       9
5.000000e+04       9
7.500000e+06       8
3.500000e+06       8
7.000000e+06       8
1.500000e+06       8
6.000000e+06       7
2.000000e+04       7
1.250000e+06       7
2.000000e+07       7
3.000000e+05       7
7.500000e+05       7
1.500000e+07       7
5.607500e+05       7
1.300000e+06       7
8.000000e+06       7
                ... 
7.950000e+05       1
5.060000e+07       1
2.545805e+07       1
1.682250e+06       1
1.730000e+08       1
9.574341e+05       1
1.180000e+07       1
4.430000e+07       1
1.160000e+07       1
8.110000e+06       1
4.850000e+07       1
2.290000e+06       1
4.090000e+08       1
7.200000e+06       1
5.270000e+07       1
1.690000e+05       1
7.290000e+07 

In [12]:
df1.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Digg,news,60,2004.0,45000000.0,"[{'description': None, 'address1': '135 Missis..."
1,Geni,web,18,2006.0,16500000.0,"[{'description': 'Headquarters', 'address1': '..."
2,Fox Interactive Media,web,0,1979.0,0.0,"[{'description': '', 'address1': '407 N Maple ..."
3,Scribd,news,50,2007.0,25800000.0,"[{'description': 'HQ', 'address1': '539 Bryant..."
4,MeetMoi,social,15,2007.0,5580000.0,"[{'description': None, 'address1': '', 'addres..."


In [13]:
def getFirst(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = df1[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [14]:
first_office.head()

,lat,lng,oficina_principal,totalOffices
0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1
1,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1
2,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ...",1
3,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",1
4,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",1


In [26]:
df_clean = pd.concat([df1,first_office], axis=1)[["name","lng","lat", "oficina_principal","totalOffices", "category_code", "number_of_employees", "founded_year", "total_money_raised"]]

In [28]:
df_clean.head()

,name,lng,lat,oficina_principal,totalOffices,category_code,number_of_employees,founded_year,total_money_raised
0,Digg,-122.394523,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,...",1,news,60,2004.0,45000000
1,Geni,-118.393064,34.090368,"{'type': 'Point', 'coordinates': [-118.393064,...",1,web,18,2006.0,165000000
2,Fox Interactive Media,-118.394170,34.076179,"{'type': 'Point', 'coordinates': [-118.39417, ...",1,web,0,1979.0,0
3,Scribd,-122.404052,37.789634,"{'type': 'Point', 'coordinates': [-122.404052,...",1,news,50,2007.0,258000000
4,MeetMoi,-73.985506,40.757929,"{'type': 'Point', 'coordinates': [-73.985506, ...",1,social,15,2007.0,558000000


In [31]:
df_clean["total_money_raised"]=df_clean["total_money_raised"].apply(pd.to_numeric)

In [34]:
df_clean.head()

,name,lng,lat,oficina_principal,totalOffices,category_code,number_of_employees,founded_year,total_money_raised
0,Digg,-122.394523,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,...",1,news,60,2004.0,45000000.0
1,Geni,-118.393064,34.090368,"{'type': 'Point', 'coordinates': [-118.393064,...",1,web,18,2006.0,165000000.0
2,Fox Interactive Media,-118.394170,34.076179,"{'type': 'Point', 'coordinates': [-118.39417, ...",1,web,0,1979.0,0.0
3,Scribd,-122.404052,37.789634,"{'type': 'Point', 'coordinates': [-122.404052,...",1,news,50,2007.0,258000000.0
4,MeetMoi,-73.985506,40.757929,"{'type': 'Point', 'coordinates': [-73.985506, ...",1,social,15,2007.0,558000000.0


In [ ]:
df_clean.to_json('limpio.json', orient="records")

In [50]:
df_clean.to_csv('limpio.csv')

In [36]:
df1.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Digg,news,60,2004.0,45000000,"[{'description': None, 'address1': '135 Missis..."
1,Geni,web,18,2006.0,165000000,"[{'description': 'Headquarters', 'address1': '..."
2,Fox Interactive Media,web,0,1979.0,0,"[{'description': '', 'address1': '407 N Maple ..."
3,Scribd,news,50,2007.0,258000000,"[{'description': 'HQ', 'address1': '539 Bryant..."
4,MeetMoi,social,15,2007.0,558000000,"[{'description': None, 'address1': '', 'addres..."


In [47]:
df1.offices.values

array([list([{'description': None, 'address1': '135 Mississippi St', 'address2': '', 'zip_code': '94107', 'city': 'San Francisco', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.764726, 'longitude': -122.394523}]),
       list([{'description': 'Headquarters', 'address1': '9229 W. Sunset Blvd.', 'address2': '', 'zip_code': '90069', 'city': 'West Hollywood', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 34.090368, 'longitude': -118.393064}]),
       list([{'description': '', 'address1': '407 N Maple Dr', 'address2': '', 'zip_code': '90210', 'city': 'Beverly Hills', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 34.076179, 'longitude': -118.39417}]),
       ...,
       list([{'description': 'HQ', 'address1': 'Rue des Tanneurs, 60a', 'address2': '', 'zip_code': '1000', 'city': 'Brussels', 'state_code': None, 'country_code': 'BEL', 'latitude': 50.83997, 'longitude': 4.3464721}]),
       list([{'description': '', 'address1': '400 Royal Palm Way', 'address2': '', '

AttributeError: 'list' object has no attribute 'get'